In [1]:
import requests
import requests_cache
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian
import csv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# System function

In [2]:
def scroll_down(driver):
    time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

def get_elements(driver, by, name, delay=2, first=False, tries=5):
    while tries > 0:
        try:
            elem = None
            if first:
                elem = WebDriverWait(driver, delay).until(EC.presence_of_element_located((by, name)))
            else:
                elem = WebDriverWait(driver, delay).until(EC.presence_of_all_elements_located((by, name)))
            return elem
        except TimeoutException:
            tries-=1

# Get data function

In [3]:
def get_info(driver, list_users_link, users):
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    for link, id in list_users_link.items():
        driver.get(link)
        users['id'].append(id)
        name = 'infoStats__value'
        elements = get_elements(driver, By.CLASS_NAME, name)
        users['name'].append(get_elements(driver, By.CLASS_NAME, 'profileHeaderInfo__userName', first=True).text)
        attrs = ['followers', 'following', 'tracks']
        for i in range(len(attrs)):
            try:
                users[attrs[i]].append(elements[i].text)
            except:
                users[attrs[i]].append(None)
        
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

def get_users(driver, link, tracks, tracks_id, list_users_link, list_tracks_link):
    global user_id
    global track_id
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[2])
    driver.get(link)
    name = 'sc-ministats-item'
    elements = get_elements(driver, By.CLASS_NAME, name)
    if link in list_tracks_link.keys():
        tracks_id.append(str(list_tracks_link[link]))
    else:
        track_id += 1
        tracks['id'].append(track_id)
        tracks_id.append(str(track_id))
        author = get_elements(driver, By.CLASS_NAME, 'sc-link-secondary', first=True)
        link = author.get_attribute('href')
        if link in list_users_link.keys():
            tracks['author'].append(list_users_link[link])
        else:
            user_id += 1
            tracks['author'].append(user_id)
            list_users_link[link] = user_id
        try:
            tracks['title'].append(get_elements(driver, By.CLASS_NAME, 'soundTitle__title', first=True).text)
        except:
            tracks['title'].append(None)
        attrs = ['plays', 'likes', 'reposts']
        for i in range(len(attrs)):
            try:
                tracks[attrs[i]].append(elements[i].get_attribute('title').split(' ')[0])
            except:
                tracks[attrs[i]].append('0')
        tracks['release'].append(get_elements(driver, By.CLASS_NAME, 'relativeTime', first=True).text)
    driver.close()
    driver.switch_to.window(driver.window_handles[1])

def get_tracks(driver, link, tracks, tracks_id, list_users_link, list_tracks_link):
    driver.execute_script("window.open('');")
    driver.switch_to.window(driver.window_handles[1])
    driver.get(link)
    name = 'trackItem__trackTitle'
    elements = get_elements(driver, By.CLASS_NAME, name)
    cnt = 0
    # crawl 2 tracks of playlist
    for e in elements:
        cnt += 1
        link = e.get_attribute('href')
        get_users(driver, link, tracks, tracks_id, list_users_link, list_tracks_link)
        if cnt == 2:
            break
    driver.close()
    driver.switch_to.window(driver.window_handles[0])

def get_playlists(driver, elements, playlists, tracks, list_users_link, list_tracks_link):
    global playlist_id
    for e in elements:
        # pass unavailable and less than 5 tracks
        data = e.text.split('\n')
        link = e.find_elements(By.CLASS_NAME, 'soundTitle__title')[0].get_attribute('href')
        if data[0] == 'Unavailable' or not(data[-6].startswith('View') and data[-6].endswith('tracks')):
            continue
        playlist_id += 1
        tracks_id = []
        playlists['id'].append(playlist_id)
        playlists['title'].append(data[2])
        playlists['author'].append(data[1])
        playlists['likes'].append(data[-5] if data[-5] != 'Like' else '0')
        playlists['reposts'].append(data[-4] if data[-4] != 'Repost' else '0')
        playlists['release'].append(data[4])
        get_tracks(driver, link, tracks, tracks_id, list_users_link, list_tracks_link)
        playlists['tracks'].append(','.join(tracks_id))

# Execute

In [4]:
driver = webdriver.Chrome()
time.sleep(1)
cookies = False
playlists = {'id': [], 'title': [], 'author': [], 'tracks': [], 'likes': [], 'reposts': [], 'release': []}
tracks = {'id': [], 'title': [], 'author': [], 'plays': [], 'likes': [], 'reposts': [], 'release': []}
users = {'id': [], 'name': [], 'followers': [], 'following': [], 'tracks': []}
global playlist_id
global track_id
global user_id
playlist_id, track_id, user_id = 0, 0, 0
list_tracks_link, list_users_link = {}, {}
cookies_id = 'onetrust-accept-btn-handler'
for i in range(ord('a'), ord('z')+1):
    elements = []
    key = chr(i)
    driver.get(f'https://soundcloud.com/search/sets?q={key}')
    if not(cookies):
        get_elements(driver, By.ID, cookies_id, delay=5, first=True).click()
        cookies = True
    while len(elements) < 50:
        scroll_down(driver)
        elements = driver.find_elements(By.CLASS_NAME, 'sound__content')
    get_playlists(driver, elements, playlists, tracks, list_users_link, list_tracks_link)
    get_info(driver, list_users_link, users)
playlist_df = pd.DataFrame(playlists, columns=['id', 'title', 'author', 'tracks', 'likes', 'reposts', 'release'])
track_df = pd.DataFrame(tracks, columns=['id', 'title', 'author', 'plays', 'likes', 'reposts', 'release'])
user_df = pd.DataFrame(users, columns=['id', 'name', 'followers', 'following', 'tracks'])
# Export to file
playlist_df.to_csv('playlists.csv', index='id')
track_df.to_csv('tracks.csv', index='id')
user_df.to_csv('users.csv', index='id')
driver.quit()
